## Project: Personal Chef

"""

1. Take the left over ingredients left over in the kitchen.
2. Searches the web for recipes that use those ingredients and return them.
3. And can answer any follow up questions.

"""

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.jpg', multiple=False)
display(uploader)

FileUpload(value=(), accept='.jpg', description='Upload')

In [10]:
print(uploader.value)

({'name': 'IMG_0916.jpg', 'type': 'image/jpeg', 'size': 1489060, 'content': <memory at 0x11227f580>, 'last_modified': datetime.datetime(2025, 12, 21, 20, 6, 15, 754000, tzinfo=datetime.timezone.utc)},)


In [11]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [12]:
from tavily import TavilyClient
from typing import Dict, Any
from langchain.tools import tool
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:
    """ Performs web search on ingredients from image """
    
    tavily_client.search(query)

In [13]:
from langchain.messages import HumanMessage

from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model = "gpt-5-nano",
    system_prompt = "Searches the web for recipes that use those ingredients and return them. Also, please be prepared to answer any follow up questions.",
    checkpointer = InMemorySaver(),
    tools = [web_search]
)

config = {"configurable": {"thread_id": "1"}}


multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me how can I use the ingredients present in this image, meaning for which recipes."},
    {"type": "image", "base64": img_b64, "mime_type": "image/jpg"}
])

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

full_content = response['messages'][-1].content
print(full_content[:3000])  # prints only the first 1000 characters
print("... [truncated]")


Nice—I can help with recipe ideas that use what I can see in the image. Here’s what I can identify and some quick ideas to get you started:

What I can identify in the fridge image
- Eggs (two cartons on the top shelf)
- A container of almonds (or some kind of nuts) on the middle shelf
- A bottle/jug of milk or similar dairy on the bottom shelf
- A few boxed sweets/chocolates and maybe a few fruits or jars (not clear)

Recipe ideas using those ingredients
- Breakfast/egg dishes
  - Classic French toast (eggs + milk + bread; you can add a pinch of cinnamon)
  - Scrambled eggs or an omelet with a splash of milk for creaminess
  - Simple egg skillet or frittata: eggs + milk, plus whatever veggies you have

- Baking with almonds and eggs
  - Almond cookies or almond flour cookies (use ground almonds/Nutty flour plus eggs)
  - Almond milk pancake or crepe batter (eggs + milk + almond flour or ground almonds)
  - Egg-based custard or pudding made with milk and a bit of almond extract if you 

In [ ]:
question = HumanMessage(content = "What is the Almond-Crusted Baked Eggs exactly? Is it to be included with boiled eggs or srambled eggs? Which one is your personal favorite?")
response = agent.invoke(
    {"messages": [question]},
    config
)
# from pprint import pprint
# pprint(response)